In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import src.preprocessing as prepro
import src.data_handler as data_handler
from src.models import LinealReg

raw_data : data_handler.RawData = data_handler.RawData()

In [2]:
engineered_data : data_handler.ProcessedData = data_handler.ProcessedData(correct_data_types=False, standarize=False, area_units='m2')
engineered_data.fill_missing_values(method='mean')
engineered_data.casas_dev['high_latitude'] = np.where(engineered_data.casas_dev['lat'] > 0, True, False)
engineered_data.casas_dev['rooms_per_area'] = engineered_data.casas_dev['rooms'] / engineered_data.casas_dev['area']
engineered_data.casas_dev['log_area'] = np.log(engineered_data.casas_dev['area'])
engineered_data.casas_dev = engineered_data.casas_dev.drop(columns=['lon', 'lat', 'area_units'])
engineered_data.casas_dev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   area            900 non-null    float64
 1   is_house        900 non-null    int64  
 2   has_pool        900 non-null    int64  
 3   age             900 non-null    float64
 4   price           900 non-null    float64
 5   rooms           900 non-null    float64
 6   high_latitude   900 non-null    bool   
 7   rooms_per_area  900 non-null    float64
 8   log_area        900 non-null    float64
dtypes: bool(1), float64(6), int64(2)
memory usage: 57.3 KB


In [3]:
train : pd.DataFrame
validation : pd.DataFrame
train, validation = data_handler.get_train_and_validation_sets(engineered_data.casas_dev, train_fraction=0.8, seed=42)
train_standarized : pd.DataFrame = prepro.standarize_numeric_columns(train, excluded_columns=set(['lat', 'lon', 'is_house', 'has_pool', 'area_units', 'high_latitude']))
validation_standarized : pd.DataFrame = prepro.standarize_numeric_columns(validation, excluded_columns=set(['lat', 'lon', 'is_house', 'has_pool', 'area_units', 'high_latitude']))
train_standarized.head()

,area,is_house,has_pool,age,price,rooms,high_latitude,rooms_per_area,log_area
70,-0.454611,0,0,0.191565,0.469669,-0.828209,True,-0.846953,-0.326818
827,0.173135,0,0,-0.709917,0.880550,0.271995,True,0.132921,0.323658
231,1.395587,0,0,-0.033806,-0.734041,1.372199,False,-0.015283,1.295449
588,-0.751964,1,1,-0.033806,0.819786,-0.828209,True,-0.277259,-0.690403
39,1.957254,1,0,1.318418,-0.563323,1.372199,False,-0.610197,1.656530


In [4]:
test_set : pd.DataFrame = raw_data.casas_test
test_set = prepro.convert_area_units(test_set, area_unit='m2')
test_set = prepro.remove_na_rows(test_set)
test_set = prepro.correct_data_types(test_set)
test_set = prepro.standarize_numeric_columns(test_set, excluded_columns=set(['area_units', 'is_house', 'has_house']))
# test_set = test_set.drop(columns=['area_units'])

In [5]:
test_set['high_latitude'] = np.where(test_set['lat'] > 0, True, False)
test_set['rooms_per_area'] = test_set['rooms'] / test_set['area']
test_set['log_area'] = np.log(test_set['area'])
test_set = test_set.drop(columns=['lon', 'lat', 'area_units'])
test_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 0 to 99
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   area            81 non-null     Float64
 1   is_house        81 non-null     bool   
 2   has_pool        81 non-null     float64
 3   age             81 non-null     Float64
 4   price           81 non-null     Float64
 5   rooms           81 non-null     Float64
 6   high_latitude   81 non-null     bool   
 7   rooms_per_area  81 non-null     Float64
 8   log_area        81 non-null     Float64
dtypes: Float64(6), bool(2), float64(1)
memory usage: 5.7 KB


/usr/local/lib/python3.12/dist-packages/pandas/core/arrays/masked.py:672: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs2, **kwargs)


In [6]:
lin : LinealReg = LinealReg(train_standarized.drop(columns='price').to_numpy(), train_standarized['price'].to_numpy(), L2=3)
lin.fit_pseudo_inverse()
print(lin.error_cuadratico_medio(validation_set_x=test_set.drop(columns=['price']).to_numpy(), validation_set_y=test_set['price'].to_numpy()))

nan
